In [4]:
import tensorflow

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

tags = []
words = []
classes = []
documents = []
ignore_words = ['?',"!"]
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aniket\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aniket\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

20 documents
6 classes ['goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'thanks']
33 unique lemmatized words ["'s", 'accept', 'anyone', 'are', 'bye', 'card', 'cash', 'credit', 'day', 'do', 'good', 'goodbye', 'hello', 'helpful', 'hi', 'hour', 'how', 'is', 'later', 'mastercard', 'only', 'open', 'see', 'take', 'thank', 'thanks', 'that', 'there', 'today', 'what', 'when', 'you', 'your']


In [11]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
    

Training data created


<ipython-input-11-c8d463de42fc>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [12]:
# create model - 3layers
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

#compile the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=["accuracy"])

#fitting and saving the model
history = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5,verbose=1)
model.save('chatbot_model.h5', history)
print("Model Created")

C:\Users\Aniket\anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
4/4 [==============================] - 1s 4ms/step - loss: 1.8119 - accuracy: 0.1500
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 1.7706 - accuracy: 0.2500
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 1.8024 - accuracy: 0.1500
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 1.7400 - accuracy: 0.1500
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.7168 - accuracy: 0.3500
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 1.6770 - accuracy: 0.4000
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 1.5389 - accuracy: 0.4000
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 1.5344 - accuracy: 0.4500
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 1.5646 - accuracy: 0.3500
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 1.3785 - accuracy: 0.5500
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 3ms/step - loss: 0.0507 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1091 - accuracy: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0334 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1242 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0680 - accuracy: 0.9500
Epoch 89/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1755 - accuracy: 1.0000
Epoch 90/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0533 - accuracy: 1.0000
Epoch 91/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0555 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0600 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0343 - accuracy: 1.0000
Epoch 94/200
4/4 [===============

4/4 [==============================] - 0s 4ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0739 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0332 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0508 - accuracy: 0.9500
Epoch 171/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0396 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0228 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0452 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0519 - accuracy: 1.0000
Epoch 176/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 177/200
4/4 [=====